In [8]:
import numpy as np
from torch import nn
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from typing import Tuple
from torch.optim import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
n = 10000
df = pd.DataFrame(
    {
        "feature1": np.random.uniform(-100,100,size = n),
    }    
)
df["feature1_first_derivative"]= df["feature1"].diff(1).fillna(0)
df["feature1_second_derivative"]= df["feature1_first_derivative"].diff(1).fillna(0)
df["feature2"]= np.random.uniform(-100,100,size = n)
df["random"] = np.random.uniform(-100,100,size = n)
df["target"] = 2*df["feature1"] + df["feature2"]  + 5
df

In [ ]:
scaler = StandardScaler().set_output(transform = "pandas")
df = scaler.fit_transform(df)
df

In [11]:
class LstmModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=2,
            hidden_size=8,
            num_layers=2,
            batch_first=True,
        )
        self.fc1 = nn.Linear(8, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y_lstm, (hn, cn) = self.lstm(x)
        # Use the output of the last time step
        y_final = self.fc1(y_lstm[:, -1, :])
        return y_final
       
class LstmDataset(Dataset):
    def __init__(
            self, 
            X_lstm: torch.Tensor,
            y_lstm: torch.Tensor,
    ) -> None:
        self.X_lstm = X_lstm
        self.y_lstm = y_lstm

    def __len__(
            self,
    ) -> int:
        return self.X_lstm.shape[0]
        
    def __getitem__(
            self, 
            index
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        X = self.X_lstm[index,:,:]

        y = self.y_lstm[index]
        return X,y

In [12]:
L = 10
X_list = []
y_list = []
for i in range(df.shape[0]-L+1):
    X_list.append(
        df.iloc[i:i+L,:-1].loc[:,["feature1", "feature2"]],
    )
    y_list.append(
        df.iloc[i+L-1,-1],
    )
X_lstm = torch.Tensor(np.array(X_list)).float()
y_lstm = torch.Tensor(np.array(y_list)).float()

In [ ]:
lstm_epochs = 3000
lstm_plt_dict = {
    "x": [],
    "y": [],
}
lstm_dataset = LstmDataset(
    X_lstm=X_lstm,
    y_lstm=y_lstm,
)

lstm_dataloader = DataLoader(
    lstm_dataset,
    batch_size=10,
    shuffle=True,
)

lstm_model = LstmModel()
lstm_criterion = torch.nn.MSELoss()

lstm_optimizer = Adam(
    params = lstm_model.parameters(),
    lr = 1/1000,
    weight_decay=1e-5,

)
for epoch in range(lstm_epochs):
    for i,(lstm_X, lstm_y_true) in enumerate(lstm_dataloader):
        lstm_optimizer.zero_grad()
        lstm_y_pred = lstm_model(lstm_X)
        lstm_loss = lstm_criterion(
            lstm_y_pred,
            lstm_y_true,
        )
        lstm_loss.backward()
        lstm_optimizer.step()
        if i % 100 == 0:
            print(lstm_loss)
        lstm_plt_dict["x"].append(i*(epoch+1))
        lstm_plt_dict["y"].append(lstm_loss.item())
        
    print(f"Epoch {epoch} done.")

In [ ]:
lstm_fig, lstm_ax = plt.subplots()
lstm_ax.scatter(lstm_plt_dict["x"],lstm_plt_dict["y"])
lstm_ax.set_xlabel("iteration")
lstm_ax.set_ylabel("loss")
lstm_fig.show()


In [ ]:
def count_trainable_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_trainable_parameters(lstm_model)

In [ ]:
torch.Tensor().numel()